### 1. Import Library

In [8]:
# Import Required Libraries
from pathlib import Path
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
import cv2  # Untuk pemrosesan gambar
from sklearn.svm import SVC  # Untuk model SVM
from sklearn.metrics import accuracy_score, classification_report  # Untuk evaluasi akurasi

### 2. Load Dataset

In [2]:
# Image directories
train_dir = "drive/MyDrive/images/training/"
test_dir = "drive/MyDrive/images/test/"

def load_dataset(img_dir):
    p = Path(img_dir)
    dirs = p.glob('*')

    img_list = []

    for dir in dirs:
        label = str(dir).split('/')[-1]
        for file in dir.glob('*.jpg'):
            img = mpimg.imread(file)

            if not img is None:
                img_list.append((img, label))

    return img_list

# Load training data
train_img = load_dataset(train_dir)

# Load test data
test_img = load_dataset(test_dir)

### 3. Preprocessing: Resize gambar dan encode label

In [3]:
def standarized_input(image):
    # Resize to width: 1100, height: 600
    std_img = cv2.resize(image, (1100, 600))
    return std_img

def label_encoder(label):
    # Encode the label: day as 1; night as 0
    return 1 if label == 'day' else 0

def preprocess(img_list):
    std_img_list = []
    for item in img_list:
        image = item[0]
        label = item[1]

        # Standarize the image
        std_img = standarized_input(image)

        # Create the label
        img_label = label_encoder(label)

        std_img_list.append((std_img, img_label))

    return std_img_list

# Preprocess training and test data
train_std_img_list = preprocess(train_img)
test_std_img_list = preprocess(test_img)

### 4. Ekstraksi Fitur Histogram

In [4]:
# Fungsi untuk menghitung histogram dari gambar
def extract_histogram_feature(img_list, bins=32):
    hist_features = []
    labels = []

    for img in img_list:
        image = img[0]  # Get image
        label = img[1]  # Get label

        # Konversi gambar ke HSV
        hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

        # Menghitung histogram untuk setiap kanal (Hue, Saturation, Value)
        h_hist = cv2.calcHist([hsv_image], [0], None, [bins], [0, 256])
        s_hist = cv2.calcHist([hsv_image], [1], None, [bins], [0, 256])
        v_hist = cv2.calcHist([hsv_image], [2], None, [bins], [0, 256])

        # Normalisasi histogram
        h_hist = cv2.normalize(h_hist, h_hist).flatten()
        s_hist = cv2.normalize(s_hist, s_hist).flatten()
        v_hist = cv2.normalize(v_hist, v_hist).flatten()

        # Gabungkan semua histogram menjadi satu vektor fitur
        hist_feature = np.hstack((h_hist, s_hist, v_hist))

        # Menyimpan fitur histogram dan label
        hist_features.append(hist_feature)
        labels.append(label)

    return np.array(hist_features), np.array(labels)

# Ekstrak fitur histogram pada data training
X_train_hist, y_train_hist = extract_histogram_feature(train_std_img_list)

# Ekstrak fitur histogram pada data testing
X_test_hist, y_test_hist = extract_histogram_feature(test_std_img_list)

### 5. Latihan Model SVM

In [5]:
# Latih model SVM dengan fitur histogram
model_hist = SVC(kernel='linear')  # Kernel bisa diganti dengan 'rbf', 'poly' dll.
model_hist.fit(X_train_hist, y_train_hist)

SVC(kernel='linear')

### 6. Evaluasi dan Laporan Klasifikasi

In [9]:
# Prediksi pada data training
y_train_pred_hist = model_hist.predict(X_train_hist)

# Prediksi pada data testing
y_test_pred_hist = model_hist.predict(X_test_hist)

# Evaluasi pada data training
acc_train_hist = accuracy_score(y_train_hist, y_train_pred_hist)
print(f'Accuracy on train (Histogram features): {acc_train_hist}')

# Evaluasi pada data testing
acc_test_hist = accuracy_score(y_test_hist, y_test_pred_hist)
print(f'Accuracy on test (Histogram features): {acc_test_hist}')

# Tampilkan laporan klasifikasi pada data training
print("\nClassification Report for Training Data:")
print(classification_report(y_train_hist, y_train_pred_hist, target_names=['night', 'day']))

# Tampilkan laporan klasifikasi pada data testing
print("\nClassification Report for Test Data:")
print(classification_report(y_test_hist, y_test_pred_hist, target_names=['night', 'day']))

Accuracy on train (Histogram features): 0.996
Accuracy on test (Histogram features): 0.9875

Classification Report for Training Data:
              precision    recall  f1-score   support

       night       1.00      0.99      1.00       120
         day       0.99      1.00      1.00       130

    accuracy                           1.00       250
   macro avg       1.00      1.00      1.00       250
weighted avg       1.00      1.00      1.00       250


Classification Report for Test Data:
              precision    recall  f1-score   support

       night       0.98      1.00      0.99        80
         day       1.00      0.97      0.99        80

    accuracy                           0.99       160
   macro avg       0.99      0.99      0.99       160
weighted avg       0.99      0.99      0.99       160

